In [20]:
import pandas as pd
import numpy as np
import requests
from yahooquery import Ticker  # Step 2 (Speed)
import yfinance as yf  # Step 3 (Reliability)
import io
import json
import os
import time

In [21]:
import os
import json

# ==========================================
# CONFIGURATION
# ==========================================
# 1. FOLDER SETUP (The Fix for GitHub Portability)
DATA_FOLDER = "YfinanceDataDump"  # Relative path (creates folder in project root)

# Create the folder if it doesn't exist
if not os.path.exists(DATA_FOLDER):
    try:
        os.makedirs(DATA_FOLDER)
        print(f"Created data folder: {DATA_FOLDER}")
    except Exception as e:
        print(
            f"Warning: Could not create folder '{DATA_FOLDER}'. Saving to current directory. Error: {e}"
        )
        DATA_FOLDER = "."

# 2. FILE PATHS (Everything saves inside the folder now)
CACHE_FILE = os.path.join(DATA_FOLDER, "financial_cache_CA.json")
FORTRESS_CSV = os.path.join(DATA_FOLDER, "fortress_stocks_CA.csv")
STRONG_CSV = os.path.join(DATA_FOLDER, "strong_stocks_CA.csv")
RISKY_CSV = os.path.join(DATA_FOLDER, "risky_stocks_CA.csv")
ANALYST_CSV = os.path.join(DATA_FOLDER, "Analyst_Fortress_Picks_CA.csv")
BUFFETT_CSV = os.path.join(DATA_FOLDER, "Buffett_Value_Picks_CA.csv")
DEEPVAL_CSV = os.path.join(DATA_FOLDER, "Deep_Value_Gems_CA.csv")
# 3. UNIVERSE FILTERS
MIN_PRICE = 2.00
MIN_VOLUME = 100_000  # 100K shares/day
MIN_CAP = 50_000_000  # $50M
MIN_CURRENT_RATIO = 1.2
MAX_PE_RATIO = 100.0

# 4. SAFETY THRESHOLDS
MIN_INTEREST_COVERAGE = 1.5
MIN_ROIC = 0.05  # 5%
FORTRESS_MARGIN_THRESHOLD = 0.05  # 5%

EXCLUDED_SECTORS = ["Financial Services", "Real Estate"]
CACHE_EXPIRY_DAYS = 30


# ==========================================
# HELPER FUNCTIONS
# ==========================================
def load_cache():
    if os.path.exists(CACHE_FILE):
        try:
            with open(CACHE_FILE, "r") as f:
                return json.load(f)
        except:
            return {}
    return {}


def save_cache(cache_data):
    try:
        with open(CACHE_FILE, "w") as f:
            json.dump(cache_data, f)
    except Exception as e:
        print(f"Warning: Could not save cache: {e}")


def calculate_altman_z_yfinance(bs, fin, market_cap):
    """
    Calculates Z-Score using yfinance DataFrames.
    Formula: 1.2A + 1.4B + 3.3C + 0.6D + 1.0E
    """
    try:
        # Helper to safely get value from Series
        def get_val(df, keys):
            for k in keys:
                if k in df.index:
                    return df.loc[k].iloc[0]
            return 0

        # Map yfinance row names
        total_assets = get_val(bs, ["Total Assets"])
        total_liab = get_val(
            bs, ["Total Liabilities Net Minority Interest", "Total Liabilities"]
        )
        current_assets = get_val(bs, ["Current Assets", "Total Current Assets"])
        current_liab = get_val(bs, ["Current Liabilities", "Total Current Liabilities"])
        retained_earnings = get_val(bs, ["Retained Earnings"])

        ebit = get_val(fin, ["EBIT", "Operating Income"])
        total_revenue = get_val(fin, ["Total Revenue"])

        if total_assets == 0 or total_liab == 0:
            return 0

        # A: Working Capital / Total Assets
        A = (current_assets - current_liab) / total_assets

        # B: Retained Earnings / Total Assets
        B = retained_earnings / total_assets

        # C: EBIT / Total Assets
        C = ebit / total_assets

        # D: Market Value of Equity / Total Liabilities
        D = market_cap / total_liab

        # E: Sales / Total Assets
        E = total_revenue / total_assets

        return (1.2 * A) + (1.4 * B) + (3.3 * C) + (0.6 * D) + (1.0 * E)
    except Exception as e:
        return 0

In [22]:
# ==========================================
# STEP 1: FETCH CANADIAN UNIVERSE (ROBUST)
# ==========================================
def get_combined_universe():
    print("--- STEP 1: Fetching Canadian Universe (TSX & TSX-V) ---")
    tickers = []

    # --- METHOD 1: TMX OFFICIAL MOC LIST ---
    url_tmx = "https://www.tsx.com/files/trading/moc-eligible-stocks.txt"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    try:
        print("   -> Attempting to fetch official TMX list...")
        response = requests.get(url_tmx, headers=headers, timeout=10)
        response.raise_for_status()

        lines = response.content.decode("utf-8").split("\n")

        for line in lines:
            parts = line.strip().split()
            if len(parts) < 3:
                continue

            # PARSING LOGIC: Detect if Exchange is at the Start or End
            # Format A: "TSX    RY    ROYAL BANK..."
            # Format B: "RY     ROYAL BANK...     TSX"

            symbol = None
            exchange = None

            if parts[0] in ["TSX", "TSXV"]:
                exchange = parts[0]
                symbol = parts[1]
            elif parts[-1] in ["TSX", "TSXV"]:
                exchange = parts[-1]
                symbol = parts[0]

            if symbol and exchange:
                # Clean Symbol (Yahoo uses hyphens, TMX uses dots)
                clean_symbol = symbol.replace(".", "-")

                if exchange == "TSX":
                    tickers.append(f"{clean_symbol}.TO")
                elif exchange == "TSXV":
                    tickers.append(f"{clean_symbol}.V")

        tickers = list(set(tickers))
        print(f"   -> Success: Found {len(tickers)} stocks via TMX.")

    except Exception as e:
        print(f"   -> TMX Fetch Failed ({e}). Trying Backup...")

    # --- METHOD 2: WIKIPEDIA S&P/TSX COMPOSITE (If Method 1 returns 0 or fails) ---
    if len(tickers) == 0:
        try:
            print("   -> Attempting to scrape S&P/TSX Composite from Wikipedia...")
            url_wiki = "https://en.wikipedia.org/wiki/S%26P/TSX_Composite_Index"
            dfs = pd.read_html(url_wiki)

            # The constituent table is usually the first or second table
            for df in dfs:
                if "Symbol" in df.columns:
                    # Wikipedia symbols often look like "RY" or "RY.TO"
                    wiki_tickers = df["Symbol"].astype(str).tolist()
                    for t in wiki_tickers:
                        t = t.replace(".", "-")  # Fix BAM.A to BAM-A
                        if not t.endswith(".TO"):
                            t = t + ".TO"
                        tickers.append(t)
                    break

            tickers = list(set(tickers))
            print(f"   -> Success: Found {len(tickers)} stocks via Wikipedia.")

        except Exception as e:
            print(f"   -> Wikipedia Scraping Failed ({e}).")

    # --- METHOD 3: EMERGENCY FALLBACK LIST ---
    if len(tickers) == 0:
        print("   -> All web fetches failed. Using Emergency Hardcoded List.")
        tickers = [
            "SHOP.TO",
            "RY.TO",
            "TD.TO",
            "CNR.TO",
            "CP.TO",
            "CSU.TO",
            "ATD.TO",
            "DOL.TO",
            "BMO.TO",
            "BNS.TO",
            "TRP.TO",
            "ENB.TO",
            "CNQ.TO",
            "BCE.TO",
            "CM.TO",
            "MFC.TO",
            "QSR.TO",
            "GIB-A.TO",
            "SU.TO",
            "WCN.TO",
            "TECK-B.TO",
            "T.TO",
            "POW.TO",
            "CVE.TO",
            "NA.TO",
            "FTS.TO",
            "EMA.TO",
            "AEM.TO",
            "WPM.TO",
            "MRU.TO",
            "OTEX.TO",
            "SAP.TO",
            "L.TO",
            "WN.TO",
            "RCI-B.TO",
            "CTC-A.TO",
            "MG.TO",
            "FM.TO",
            "K.TO",
            "CAE.TO",
            "TIH.TO",
            "GIL.TO",
            "DOO.TO",
            "STN.TO",
            "EFN.TO",
            "KEY.TO",
            "PPL.TO",
            "IMO.TO",
        ]
        print(f"   -> Loaded {len(tickers)} major stocks.")

    return tickers

In [23]:
# ==========================================
# STEP 2: LIGHTWEIGHT SIEVE (YahooQuery)
# ==========================================
def get_initial_survivors(tickers):
    print(f"\n--- STEP 2: Running 'Lightweight' Filter on {len(tickers)} stocks ---")
    chunk_size = 500
    survivors = []
    chunks = [tickers[i : i + chunk_size] for i in range(0, len(tickers), chunk_size)]

    for i, chunk in enumerate(chunks):
        if i % 2 == 0:
            print(f" -> Processing Batch {i+1}/{len(chunks)}...")
        try:
            yq = Ticker(chunk, asynchronous=True)
            df_modules = yq.get_modules(
                "summaryProfile summaryDetail financialData price defaultKeyStatistics"
            )

            for symbol, data in df_modules.items():
                if isinstance(data, str):
                    continue
                try:
                    price = data.get("price", {}).get("regularMarketPrice", 0)
                    if price is None:
                        price = 0

                    vol = data.get("summaryDetail", {}).get("averageVolume", 0)
                    if vol is None or vol == 0:
                        vol = data.get("price", {}).get("averageDailyVolume10Day", 0)

                    cap = data.get("price", {}).get("marketCap", 0)
                    if cap is None:
                        cap = 0

                    sector = data.get("summaryProfile", {}).get("sector", "Unknown")
                    fin_data = data.get("financialData", {})
                    curr_ratio = fin_data.get("currentRatio", 0)
                    op_margins = fin_data.get("operatingMargins", 0)
                    if curr_ratio is None:
                        curr_ratio = 0
                    if op_margins is None:
                        op_margins = 0

                    # --- P/E RATIO CHECK ---
                    pe = data.get("summaryDetail", {}).get("trailingPE")
                    if pe is not None and pe > MAX_PE_RATIO:
                        continue

                    # FILTERS
                    if price < MIN_PRICE:
                        continue
                    if cap < MIN_CAP:
                        continue
                    if vol < MIN_VOLUME:
                        continue
                    if any(x in sector for x in EXCLUDED_SECTORS):
                        continue
                    if curr_ratio < MIN_CURRENT_RATIO:
                        continue
                    if op_margins <= 0:
                        continue

                    survivors.append(
                        {
                            "Ticker": symbol,
                            "Sector": sector,
                            "Price": price,
                            "Op Margin %": round(op_margins * 100, 2),
                            "P/E": round(pe, 2) if pe else 0,
                            "Curr Ratio": curr_ratio,
                            "Mkt Cap (B)": round(cap / 1_000_000_000, 2),
                        }
                    )
                except:
                    continue
        except:
            continue
    return pd.DataFrame(survivors)

In [24]:
# ==========================================
# STEP 3: DEEP DIVE (yfinance + Cache)
# ==========================================
def get_advanced_metrics(survivor_df):
    tickers = survivor_df["Ticker"].tolist()
    print(f"\n--- STEP 3: Fetching Deep Financials for {len(tickers)} Survivors ---")

    cache = load_cache()
    current_time = time.time()
    expiry_seconds = CACHE_EXPIRY_DAYS * 86400

    final_data = []

    for i, ticker in enumerate(tickers):
        if i % 20 == 0:
            print(f" -> Analyzing {i+1}/{len(tickers)}: {ticker}...")

        # --- CRITICAL FIX: Anti-Throttle Sleep ---
        # time.sleep(0.75)  # Sleep to avoid hitting Yahoo too fast change back to 1 second if throttling occurs
        # -----------------------------------------

        # Helper: Logic to assign Tier based on Average Margin & Safety
        def determine_tier_history(metrics, is_fortress_margin, is_pos_margin):
            # 1. Safety Checks (Must pass these regardless of margins)
            if metrics["int_cov"] < MIN_INTEREST_COVERAGE:
                return "Risky"
            if metrics["roic"] < MIN_ROIC:
                return "Risky"

            # 2. Historical Margin Checks (Using the 4-Year Average)
            if is_fortress_margin:
                return "Fortress"  # Avg Margin > 5%
            elif is_pos_margin:
                return "Strong"  # Avg Margin > 0%

            return "Risky"  # Avg Margin was negative

        # 1. CHECK CACHE
        cached_data = cache.get(ticker)
        if cached_data and (current_time - cached_data["timestamp"] < expiry_seconds):
            if cached_data.get("roic") == -999:
                continue
            # If using cache, we might miss the 'avg_margin' recalculation unless we force update.
            # ideally we proceed to fetch if we suspect cache is old logic, but for now we trust cache.
            # To force new logic, clear your cache file (delete financial_cache.json).
            pass

        # 2. FETCH NEW DATA
        try:
            stock = yf.Ticker(ticker)
            fin = stock.financials
            bs = stock.balance_sheet

            # Check if Yahoo actually gave us data
            if fin.empty or bs.empty:
                # Don't cache this as a failure immediately; it might be a connection blip.
                # But to keep logic simple, we skip.
                print(f"   ⚠️ No data for {ticker} (skipping)")
                continue

            # --- A. NEW LOGIC: 4-Year Average Margin Check ---
            try:
                # Get Operating Income (try 'Operating Income' first, then 'EBIT')
                if "Operating Income" in fin.index:
                    op_income_history = fin.loc["Operating Income"]
                elif "EBIT" in fin.index:
                    op_income_history = fin.loc["EBIT"]
                else:
                    op_income_history = pd.Series([0])

                # Get Revenue
                revenue_history = fin.loc["Total Revenue"]

                # Calculate Margins for every available year
                # This automatically handles 1, 2, 3, or 4 years of data
                yearly_margins = (op_income_history / revenue_history).dropna()

                if len(yearly_margins) > 0:
                    avg_margin = yearly_margins.mean()

                    # The Uniform Rule: Is the AVERAGE above the threshold?
                    is_fortress_margin = avg_margin > FORTRESS_MARGIN_THRESHOLD
                    is_positive_margin = avg_margin > 0
                else:
                    is_fortress_margin = False
                    is_positive_margin = False

            except Exception as e:
                # Fail safe
                is_fortress_margin = False
                is_positive_margin = False
            # ---------------------------------------------------

            # --- B. Standard Calculations (Safety Checks) ---
            def get_item(df, keys):
                for k in keys:
                    if k in df.index:
                        return df.loc[k].iloc[0]
                return 0

            ebit = get_item(fin, ["EBIT", "Operating Income", "Pretax Income"])
            int_exp = get_item(
                fin, ["Interest Expense", "Interest Expense Non Operating"]
            )
            total_assets = get_item(bs, ["Total Assets"])
            curr_liab = get_item(
                bs, ["Current Liabilities", "Total Current Liabilities"]
            )

            # Interest Coverage
            int_exp = abs(int_exp)
            if int_exp == 0:
                int_cov = 100
            else:
                int_cov = ebit / int_exp

            # ROIC
            invested_cap = total_assets - curr_liab
            if invested_cap <= 0:
                roic = 0
            else:
                roic = ebit / invested_cap

            # Z-Score
            base_row = survivor_df[survivor_df["Ticker"] == ticker].iloc[0]
            mkt_cap_raw = base_row["Mkt Cap (B)"] * 1_000_000_000
            z = calculate_altman_z_yfinance(bs, fin, mkt_cap_raw)

            # Cache the metrics
            metrics = {
                "timestamp": current_time,
                "z_score": round(z, 2),
                "roic": roic,
                "int_cov": round(int_cov, 2),
            }
            cache[ticker] = metrics

            # --- C. Determine Final Tier ---
            tier = determine_tier_history(
                metrics, is_fortress_margin, is_positive_margin
            )

            final_data.append(
                {
                    "Ticker": ticker,
                    "Tier": tier,
                    "Price": base_row["Price"],
                    "P/E": base_row["P/E"],
                    "Sector": base_row["Sector"],
                    "Z-Score": round(z, 2),
                    "ROIC %": round(roic * 100, 2),
                    "Op Margin %": base_row["Op Margin %"],
                    "Avg Margin (4Y)": (
                        round(avg_margin * 100, 2) if "avg_margin" in locals() else 0
                    ),
                    "Curr Ratio": base_row["Curr Ratio"],
                    "Int Cov": round(int_cov, 2),
                    "Mkt Cap (B)": base_row["Mkt Cap (B)"],
                }
            )

        except Exception as e:
            continue

    save_cache(cache)
    return pd.DataFrame(final_data)

In [25]:
# ==========================================
# MAIN EXECUTION
# ==========================================
if __name__ == "__main__":
    tickers = get_combined_universe()

    if len(tickers) > 0:
        survivors_df = get_initial_survivors(tickers)

        if not survivors_df.empty:
            print(
                f"\n✅ Step 2 Complete. {len(survivors_df)} stocks passed basic filters."
            )

            final_results = get_advanced_metrics(survivors_df)

            if not final_results.empty:
                final_results = final_results.sort_values(
                    by=["Tier", "Z-Score"], ascending=[True, False]
                )

                # 1. Standard Split
                fortress_df = final_results[final_results["Tier"] == "Fortress"].copy()
                strong_df = final_results[final_results["Tier"] == "Strong"].copy()
                risky_df = final_results[final_results["Tier"] == "Risky"].copy()

                # 3. Save Files (Updated to use Relative Paths from Cell 2)
                try:
                    fortress_df.to_csv(FORTRESS_CSV, index=False)
                    strong_df.to_csv(STRONG_CSV, index=False)
                    risky_df.to_csv(RISKY_CSV, index=False)

                    print("\n" + "=" * 60)
                    print("RESULTS GENERATED")
                    print("=" * 60)
                    print(
                        f"1. FORTRESS ({len(fortress_df)}): Saved to '{FORTRESS_CSV}'"
                    )
                    print(f"2. STRONG   ({len(strong_df)}): Saved to '{STRONG_CSV}'")
                    print(f"3. RISKY    ({len(risky_df)}): Saved to '{RISKY_CSV}'")
                except Exception as e:
                    print(f"\n⚠️ Error Saving Files: {e}")
                    print("Check if the file is open in Excel or if the folder exists.")

                pd.set_option("display.max_rows", 500)
                pd.set_option("display.max_columns", 20)
                pd.set_option("display.width", 1000)

                print("\n--- FORTRESS PREVIEW ---")
                print(fortress_df.head(15))
            else:
                print("No stocks passed the deep financial analysis.")
        else:
            print("No stocks passed the initial lightweight filter.")
    else:
        print("Could not fetch ticker universe.")

--- STEP 1: Fetching Canadian Universe (TSX & TSX-V) ---
   -> Attempting to fetch official TMX list...
   -> Success: Found 1088 stocks via TMX.

--- STEP 2: Running 'Lightweight' Filter on 1088 stocks ---
 -> Processing Batch 1/3...
 -> Processing Batch 3/3...

✅ Step 2 Complete. 111 stocks passed basic filters.

--- STEP 3: Fetching Deep Financials for 111 Survivors ---
 -> Analyzing 1/111: MDI.TO...
   ⚠️ No data for RGSI.TO (skipping)
 -> Analyzing 21/111: CMG.TO...
 -> Analyzing 41/111: FAR.TO...
 -> Analyzing 61/111: TI.TO...
   ⚠️ No data for CPKR.TO (skipping)
 -> Analyzing 81/111: ALS.TO...
 -> Analyzing 101/111: MFI.TO...

RESULTS GENERATED
1. FORTRESS (74): Saved to 'YfinanceDataDump\fortress_stocks_CA.csv'
2. STRONG   (5): Saved to 'YfinanceDataDump\strong_stocks_CA.csv'
3. RISKY    (30): Saved to 'YfinanceDataDump\risky_stocks_CA.csv'

--- FORTRESS PREVIEW ---
      Ticker      Tier   Price    P/E             Sector  Z-Score  ROIC %  Op Margin %  Avg Margin (4Y)  Curr Rat

In [26]:
# 1. Define the function (if you haven't already in a previous cell)
def get_analyst_fortress_from_var(df_input):
    working_df = df_input.copy()
    tickers = working_df["Ticker"].tolist()

    print(
        f"\n--- STEP 4: Fetching Analyst Ratings for {len(tickers)} Stocks (From Memory) ---"
    )
    print("    (Fetching serially to avoid throttling...)")

    analyst_data = []

    for i, ticker in enumerate(tickers):
        if i % 10 == 0:
            print(f" -> Analyst Scan {i+1}/{len(tickers)}: {ticker}...")

        try:
            stock = yf.Ticker(ticker)
            info = stock.info

            rec_mean = info.get("recommendationMean")
            target_price = info.get("targetMeanPrice")
            current_price = info.get("currentPrice")

            # Filter: Must be better than 2.0 (Lower is better)
            if rec_mean is None or rec_mean > 2.0:
                continue

            upside = 0
            if target_price and current_price:
                upside = round(
                    ((target_price - current_price) / current_price) * 100, 2
                )

            # Merge with existing data
            base_row = working_df[working_df["Ticker"] == ticker].iloc[0].to_dict()
            base_row["Analyst_Rating"] = rec_mean
            base_row["Target_Price"] = target_price
            base_row["Upside_%"] = upside

            analyst_data.append(base_row)
            time.sleep(0.2)  # Polite delay

        except Exception:
            continue

    return pd.DataFrame(analyst_data)


# ==========================================
# 2. EXECUTE IT (Run this part!)
# ==========================================

# Check if fortress_df exists from the previous step
if "fortress_df" in locals() and not fortress_df.empty:

    # Run the function
    Analyst_Fortress_DF = get_analyst_fortress_from_var(fortress_df)

    if not Analyst_Fortress_DF.empty:
        # Sort by best Analyst Rating (Lower is better) or Upside
        Analyst_Fortress_DF = Analyst_Fortress_DF.sort_values(
            by="Upside_%", ascending=False
        )

        # Display Results
        print("\n✅ Analyst Scan Complete!")
        print(f"Found {len(Analyst_Fortress_DF)} stocks with Buy Ratings (Score < 2.0)")

        # Save to CSV
        Analyst_Fortress_DF.to_csv(ANALYST_CSV, index=False)
        print("Saved to 'Analyst_Fortress_Picks.csv'")

        # Show top picks
        cols = ["Ticker", "Price", "Analyst_Rating", "Upside_%", "Target_Price", "Tier"]
        print(Analyst_Fortress_DF[cols].head(20))
    else:
        print("No stocks passed the Analyst filter.")
else:
    print(
        "❌ 'fortress_df' not found or empty. Please run the Main Filter (Step 1-3) first."
    )


--- STEP 4: Fetching Analyst Ratings for 74 Stocks (From Memory) ---
    (Fetching serially to avoid throttling...)
 -> Analyst Scan 1/74: WPM.TO...
 -> Analyst Scan 11/74: DNG.TO...
 -> Analyst Scan 21/74: K.TO...
 -> Analyst Scan 31/74: RUS.TO...
 -> Analyst Scan 41/74: IMO.TO...
 -> Analyst Scan 51/74: POU.TO...
 -> Analyst Scan 61/74: WN.TO...
 -> Analyst Scan 71/74: CTC-A.TO...

✅ Analyst Scan Complete!
Found 30 stocks with Buy Ratings (Score < 2.0)
Saved to 'Analyst_Fortress_Picks.csv'
     Ticker   Price  Analyst_Rating  Upside_%  Target_Price      Tier
17   VNP.TO   17.72         1.50000     43.04     25.346025  Fortress
21   CVE.TO   23.22         1.52941     27.81     29.676470  Fortress
3    TXG.TO   65.54         1.50000     26.58     82.958330  Fortress
24   PBH.TO  101.71         1.72727     25.04    127.181820  Fortress
20   PHX.TO    7.50         1.66667     22.22      9.166670  Fortress
6    AGI.TO   53.00         1.38462     21.09     64.176130  Fortress
23   DOO.TO 

In [27]:
import pandas as pd
from yahooquery import Ticker


# ==========================================
# BUFFETT "BELOW NAV" SCAN
# ==========================================
def get_buffett_value_picks(df_input):
    print(f"\n--- STEP 5: Warren Buffett 'Below NAV' Scan ---")
    print(f"    Scanning {len(df_input)} candidates for Deep Value...")
    print(
        "    Criteria: P/B < 1.0 (Below Book) | ROE > 0% (Profitable) | Debt/Eq < 100%"
    )

    tickers = df_input["Ticker"].tolist()
    buffett_candidates = []

    # Use YahooQuery for speed (Key Stats are summary data, no throttling risk here)
    chunk_size = 250
    chunks = [tickers[i : i + chunk_size] for i in range(0, len(tickers), chunk_size)]

    for chunk in chunks:
        try:
            yq = Ticker(chunk, asynchronous=True)
            # We need defaultKeyStatistics (P/B) and financialData (ROE, Debt)
            data = yq.get_modules("defaultKeyStatistics financialData")

            for symbol in chunk:
                if isinstance(data, dict) and symbol in data:
                    try:
                        stats = data[symbol].get("defaultKeyStatistics", {})
                        fin = data[symbol].get("financialData", {})

                        # 1. Price to Book < 1.0 (The Core Rule)
                        pb = stats.get("priceToBook")
                        # Skip if None, > 1.0, or negative (insolvent)
                        if pb is None or pb >= 1.0 or pb <= 0:
                            continue

                        # 2. Positive ROE (No Zombies)
                        roe = fin.get("returnOnEquity", 0)
                        if roe is None or roe <= 0:
                            continue

                        # 3. Reasonable Debt (Safety)
                        # Buffett hates high leverage on weak companies
                        de = fin.get("debtToEquity", 0)
                        if de is None or de > 100:
                            continue

                        # Get base data from input_df
                        base_row = (
                            df_input[df_input["Ticker"] == symbol].iloc[0].to_dict()
                        )

                        # Add new Value Metrics
                        base_row["P/B Ratio"] = round(pb, 2)
                        base_row["ROE %"] = round(roe * 100, 2)
                        base_row["Debt/Eq %"] = round(de, 2)

                        buffett_candidates.append(base_row)

                    except:
                        continue
        except:
            continue

    return pd.DataFrame(buffett_candidates)


# ==========================================
# EXECUTION BLOCK
# ==========================================
# Ensure we have the 'final_results' from the Main Filter
if "final_results" in locals() and not final_results.empty:

    Buffett_Value_DF = get_buffett_value_picks(final_results)

    if not Buffett_Value_DF.empty:
        # Sort by P/B Ratio (Cheapest first)
        Buffett_Value_DF = Buffett_Value_DF.sort_values(by="P/B Ratio", ascending=True)

        # Save results
        Buffett_Value_DF.to_csv(BUFFETT_CSV, index=False)

        print("\n" + "=" * 60)
        print("BUFFETT SCAN COMPLETE")
        print("=" * 60)
        print(f"Found {len(Buffett_Value_DF)} Deep Value Stocks (Trading < Book Value)")
        print("Saved to: 'Buffett_Value_Picks.csv'")

        # Display
        pd.set_option("display.max_rows", 500)
        cols = ["Ticker", "Price", "P/B Ratio", "ROE %", "Debt/Eq %", "Sector", "Tier"]
        print("\n--- DEEP VALUE PICKS ---")
        print(Buffett_Value_DF[cols].head(20))

    else:
        print(
            "\n❌ No stocks passed the Buffett Value filter (All stocks are trading > Book Value)."
        )
else:
    print("❌ 'final_results' variable not found. Please run the Main Filter first.")


--- STEP 5: Warren Buffett 'Below NAV' Scan ---
    Scanning 109 candidates for Deep Value...
    Criteria: P/B < 1.0 (Below Book) | ROE > 0% (Profitable) | Debt/Eq < 100%

BUFFETT SCAN COMPLETE
Found 2 Deep Value Stocks (Trading < Book Value)
Saved to: 'Buffett_Value_Picks.csv'

--- DEEP VALUE PICKS ---
     Ticker  Price  P/B Ratio  ROE %  Debt/Eq %             Sector      Tier
1   MATR.TO   7.98       0.64   3.66      80.14             Energy     Risky
0  TCL-A.TO  22.72       0.99   8.94      42.46  Consumer Cyclical  Fortress


In [28]:
import pandas as pd
from yahooquery import Ticker
from IPython.display import display, Markdown
import warnings

# Ignore these specific FutureWarning messages
warnings.simplefilter(action="ignore", category=FutureWarning)


# ==========================================
# INSIDER FILTER FUNCTION (With Price)
# ==========================================
def filter_for_insider_buying(tickers):
    print(f"🕵️ Scanning {len(tickers)} stocks for Insider Buying & Price...")
    insider_picks = []

    # Chunk to prevent timeouts
    chunk_size = 20
    chunks = [tickers[i : i + chunk_size] for i in range(0, len(tickers), chunk_size)]

    for chunk in chunks:
        try:
            # Initialize Ticker object for the chunk
            yq = Ticker(chunk, asynchronous=True)

            # 1. Fetch Insider Transactions
            df_insiders = yq.insider_transactions

            # 2. Fetch Price Data (New Step)
            # This returns a dictionary: {'TICKER': {'regularMarketPrice': 10.50, ...}}
            price_data = yq.price

            # Validation: Ensure we have data to work with
            if isinstance(df_insiders, dict) or not hasattr(df_insiders, "reset_index"):
                continue

            df_insiders = df_insiders.reset_index()

            for symbol in chunk:
                if symbol not in df_insiders["symbol"].values:
                    continue

                # --- INSIDER LOGIC ---
                stock_tx = df_insiders[df_insiders["symbol"] == symbol].copy()

                buys = stock_tx[
                    stock_tx["transactionText"]
                    .astype(str)
                    .str.contains("Purchase", case=False, na=False)
                ]
                sells = stock_tx[
                    stock_tx["transactionText"]
                    .astype(str)
                    .str.contains("Sale", case=False, na=False)
                ]

                buy_vol = buys["shares"].sum() if not buys.empty else 0
                sell_vol = sells["shares"].sum() if not sells.empty else 0

                # --- PRICE LOGIC ---
                current_price = None
                try:
                    # Safely attempt to grab the price from the dictionary
                    if isinstance(price_data, dict) and symbol in price_data:
                        current_price = price_data[symbol].get(
                            "regularMarketPrice", None
                        )
                except Exception:
                    current_price = None

                # Only keep if Net Buying is Positive
                if buy_vol > sell_vol:
                    insider_picks.append(
                        {
                            "Ticker": symbol,
                            "Current_Price": current_price,  # <--- New Column
                            "Insider_Buys_Count": len(buys),
                            "Net_Shares_Bought": buy_vol - sell_vol,
                        }
                    )

        except Exception as e:
            # print(f"Error on chunk: {e}") # Uncomment for debugging
            continue

    return pd.DataFrame(insider_picks)


# ==========================================
# 2. CREATE 'Fortress_insiders' DATAFRAME
# ==========================================

# Use fortress_df if it exists, otherwise use the top 20 backup list
if "fortress_df" in locals() and not fortress_df.empty:
    target_tickers = fortress_df["Ticker"].tolist()
else:
    print("⚠️ 'fortress_df' not found or empty.")

# Run the filter
Fortress_insiders = filter_for_insider_buying(target_tickers)

# Display so Data Wrangler picks it up
print(f"✅ Created 'Fortress_insiders' with {len(Fortress_insiders)} rows.")
display(Fortress_insiders)

🕵️ Scanning 74 stocks for Insider Buying & Price...
✅ Created 'Fortress_insiders' with 43 rows.


,Ticker,Current_Price,Insider_Buys_Count,Net_Shares_Bought
0,DPM.TO,42.42,86,9142771.0
1,KNT.TO,22.69,1,8333.0
2,GRGD.TO,82.67,32,648000.0
3,WDO.TO,22.74,7,677900.0
4,DNG.TO,5.87,136,159600.0
5,TXG.TO,65.54,18,554667.0
6,ALS.TO,40.84,9,54100.0
7,ATZ.TO,117.35,62,471200.0
8,PAAS.TO,71.16,15,1368070.0
9,APM.TO,9.76,40,2101921.0


In [29]:
# ==========================================
# 2. ANALYST FILTER FUNCTION FOR INSIDER PICKS (NEW)
# ==========================================
def filter_for_analyst_ratings(df_insiders, max_score=2.5):
    """
    Fetches analyst data for the insider winners and filters for 'Buy' or better.
    Scale: 1.0 = Strong Buy, 5.0 = Sell.
    Cutoff: 2.5 ensures we get 'Buy' and 'Strong Buy'.
    """
    if df_insiders.empty:
        return df_insiders

    tickers = df_insiders["Ticker"].tolist()

    try:
        yq = Ticker(tickers, asynchronous=True)
        # 'financial_data' contains the specific recommendation scores
        fin_data = yq.financial_data

        analyst_data = []
        for t in tickers:
            # Check if we got valid data for this ticker
            if isinstance(fin_data, dict) and t in fin_data:
                data = fin_data[t]
                # Ensure it's a dictionary and has the key we need
                if isinstance(data, dict) and "recommendationMean" in data:
                    score = data.get("recommendationMean")

                    # Only keep valid scores (sometimes they are None)
                    if score is not None:
                        analyst_data.append(
                            {
                                "Ticker": t,
                                "Analyst_Score": score,
                                "Analyst_Verdict": data.get("recommendationKey", "N/A"),
                            }
                        )

        df_analyst = pd.DataFrame(analyst_data)

        if df_analyst.empty:
            print("⚠️ No Analyst ratings found for these tickers.")
            return df_insiders  # Return original if no data found

        # Merge with the Insider DataFrame
        merged = pd.merge(df_insiders, df_analyst, on="Ticker", how="inner")

        # FILTER: Keep only scores <= max_score (Lower is better)
        final_df = merged[merged["Analyst_Score"] <= max_score].copy()

        print(
            f"✅ Analyst Filter: {len(merged)} -> {len(final_df)} stocks (Min Rating: Buy)."
        )
        return final_df.sort_values(by="Analyst_Score", ascending=True)

    except Exception as e:
        print(f"❌ Error in Analyst Filter: {e}")
        return df_insiders


# ==========================================
# 3. EXECUTION PIPELINE
# ==========================================

# A. Setup Tickers
if "fortress_df" in locals() and not fortress_df.empty:
    target_tickers = fortress_df["Ticker"].tolist()
else:
    # Backup list just in case
    target_tickers = [
        "PET.TO",
        "MFI.TO",
        "TXG.TO",
        "SAP.TO",
        "PAAS.TO",
        "NEO.TO",
        "WPM.TO",
        "FNV.TO",
        "LUG.TO",
        "DPM.TO",
        "ASM.TO",
        "PNG.V",
        "DSG.TO",
        "KNT.TO",
        "GGD.TO",
        "GRGD.TO",
        "WDO.TO",
        "OGC.TO",
        "DNG.TO",
        "CLS.TO",
    ]

# B. Run Insider Filter
insider_winners = filter_for_insider_buying(target_tickers)

# C. Run Analyst Filter (NEW STEP)
# We overwrite 'Fortress_insiders' so it works with your Data Wrangler flow
if not insider_winners.empty:
    Fortress_insiders_Analyst_buy = filter_for_analyst_ratings(
        insider_winners, max_score=2.5
    )
else:
    Fortress_insiders_Analyst_buy = pd.DataFrame()

# D. Display Result
if not Fortress_insiders_Analyst_buy.empty:
    print(
        f"\n🚀 Final List: {len(Fortress_insiders_Analyst_buy)} stocks (Fortress + Insider Buying + Analyst Buy Rating)"
    )
    display(Fortress_insiders_Analyst_buy)
else:
    print("No stocks passed all filters.")

🕵️ Scanning 74 stocks for Insider Buying & Price...
✅ Analyst Filter: 26 -> 24 stocks (Min Rating: Buy).

🚀 Final List: 24 stocks (Fortress + Insider Buying + Analyst Buy Rating)


,Ticker,Current_Price,Insider_Buys_Count,Net_Shares_Bought,Analyst_Score,Analyst_Verdict
4,ATZ.TO,117.35,62,471200.0,1.42857,strong_buy
0,DPM.TO,42.42,86,9142771.0,1.50000,strong_buy
2,TXG.TO,65.54,18,554667.0,1.50000,strong_buy
13,CEU.TO,12.27,22,31620438.0,1.50000,strong_buy
20,CVE.TO,23.22,73,53965480.0,1.52941,buy
24,EFX.TO,21.16,12,4189200.0,1.55556,buy
1,GRGD.TO,82.67,32,648000.0,1.58333,buy
23,PBH.TO,101.71,7,32771.0,1.72727,buy
18,POU.TO,24.19,1,4945000.0,1.77778,buy
6,HWX.TO,9.37,6,1047100.0,1.77778,buy


In [30]:
import pandas as pd
from yahooquery import Ticker
from IPython.display import display, Markdown


# ==========================================
# BURRY "RELATIVE VALUE" FILTER (EV/EBITDA)
# ==========================================
def filter_burry_ev_ebitda(df_input):
    if df_input is None or df_input.empty:
        print("❌ Input DataFrame is empty.")
        return pd.DataFrame()

    print(f"📉 Analyzing EV/EBITDA for {len(df_input)} Fortress stocks...")

    tickers = df_input["Ticker"].tolist()

    # 1. BATCH FETCH DATA (YahooQuery for Speed)
    # We fetch multiple modules now to ensure we have ingredients for manual calc
    try:
        yq = Ticker(tickers, asynchronous=True)
        # Fetch key stats (pre-calced), financial data (debt/cash), and summary (market cap)
        data = yq.get_modules("defaultKeyStatistics financialData summaryDetail")
    except Exception as e:
        print(f"❌ Error fetching data: {e}")
        return pd.DataFrame()

    ev_data = []

    # 2. PARSE DATA
    for ticker in tickers:
        try:
            # Safe extraction of modules
            ticker_data = data.get(ticker, {})
            if isinstance(ticker_data, str):
                continue  # Handle API errors

            stats = ticker_data.get("defaultKeyStatistics", {})
            fin_data = ticker_data.get("financialData", {})
            summary = ticker_data.get("summaryDetail", {})

            # --- PLAN A: Pre-calculated Metric ---
            ev_ebitda = stats.get("enterpriseToEbitda")

            # --- PLAN B: Manual Calculation (The Fallback) ---
            if ev_ebitda is None:
                try:
                    # We need all 4 components to calculate it manually
                    market_cap = summary.get("marketCap")
                    total_debt = fin_data.get("totalDebt")
                    total_cash = fin_data.get("totalCash")
                    ebitda = fin_data.get("ebitda")

                    if all(
                        v is not None
                        for v in [market_cap, total_debt, total_cash, ebitda]
                    ):
                        if ebitda != 0:
                            # Formula: EV = Market Cap + Debt - Cash
                            enterprise_value = market_cap + total_debt - total_cash
                            ev_ebitda = enterprise_value / ebitda
                            # print(f"   -> Manual Calc success for {ticker}: {round(ev_ebitda, 2)}")
                except Exception:
                    pass  # If manual calc fails, we just skip

            # Filter: We only want profitable EBITDA for valuation (exclude negatives/None)
            if ev_ebitda is not None and ev_ebitda > 0:
                ev_data.append({"Ticker": ticker, "EV/EBITDA": round(ev_ebitda, 2)})
        except:
            continue

    df_vals = pd.DataFrame(ev_data)

    if df_vals.empty:
        print("⚠️ Could not retrieve EV/EBITDA data (even with manual fallback).")
        return pd.DataFrame()

    # 3. MERGE WITH SECTOR DATA
    # We merge back with original DF to get the 'Sector' column
    merged_df = pd.merge(df_input, df_vals, on="Ticker", how="inner")

    # 4. CALCULATE SECTOR AVERAGES
    print("\n--- 📊 SECTOR AVERAGES (EV/EBITDA) ---")
    sector_stats = merged_df.groupby("Sector")["EV/EBITDA"].mean().reset_index()
    sector_stats.rename(columns={"EV/EBITDA": "Sector_Avg_EV_EBITDA"}, inplace=True)
    sector_stats["Sector_Avg_EV_EBITDA"] = sector_stats["Sector_Avg_EV_EBITDA"].round(2)

    # Print the Benchmark Table
    print(sector_stats.to_string(index=False))

    # 5. FILTER: STOCK < SECTOR AVERAGE
    final_df = pd.merge(merged_df, sector_stats, on="Sector", how="left")

    # The Burry Filter: Value must be lower than the peer average
    burry_picks = final_df[
        final_df["EV/EBITDA"] < final_df["Sector_Avg_EV_EBITDA"]
    ].copy()

    # Calculate "Discount" metric for sorting
    burry_picks["Discount_%"] = round(
        (1 - (burry_picks["EV/EBITDA"] / burry_picks["Sector_Avg_EV_EBITDA"])) * 100, 2
    )

    # Sort by the biggest discount relative to sector
    burry_picks = burry_picks.sort_values(by="Discount_%", ascending=False)

    return burry_picks


# ==========================================
# EXECUTION
# ==========================================

# Ensure we use the fortress_df from previous steps
if "fortress_df" in locals() and not fortress_df.empty:

    Fortress_Burry_EV_EBITDA = filter_burry_ev_ebitda(fortress_df)

    if not Fortress_Burry_EV_EBITDA.empty:
        print(
            f"\n✅ Found {len(Fortress_Burry_EV_EBITDA)} Undervalued Stocks (Cheaper than Sector Avg)."
        )
        print("Created variable: 'Fortress_Burry_EV_EBITDA'")

        # Display for Data Wrangler
        display(
            Fortress_Burry_EV_EBITDA[
                [
                    "Ticker",
                    "Sector",
                    "Price",
                    "EV/EBITDA",
                    "Sector_Avg_EV_EBITDA",
                    "Discount_%",
                    "Tier",
                ]
            ]
        )
    else:
        print("No stocks found trading below their sector average.")
else:
    print("⚠️ 'fortress_df' variable not found. Please run Step 1-3 first.")

📉 Analyzing EV/EBITDA for 74 Fortress stocks...

--- 📊 SECTOR AVERAGES (EV/EBITDA) ---
            Sector  Sector_Avg_EV_EBITDA
   Basic Materials                 17.85
 Consumer Cyclical                 15.72
Consumer Defensive                 12.14
            Energy                  7.43
       Industrials                 13.60
        Technology                 17.97
         Utilities                 10.87

✅ Found 45 Undervalued Stocks (Cheaper than Sector Avg).
Created variable: 'Fortress_Burry_EV_EBITDA'


,Ticker,Sector,Price,EV/EBITDA,Sector_Avg_EV_EBITDA,Discount_%,Tier
16,MSA.TO,Basic Materials,5.62,5.84,17.85,67.28,Fortress
8,WDO.TO,Basic Materials,22.74,6.19,17.85,65.32,Fortress
32,FVI.TO,Basic Materials,13.45,6.32,17.85,64.59,Fortress
55,TCL-A.TO,Consumer Cyclical,22.72,6.58,15.72,58.14,Fortress
51,FAR.TO,Basic Materials,2.40,7.83,17.85,56.13,Fortress
37,ENGH.TO,Technology,20.36,7.96,17.97,55.70,Fortress
28,CG.TO,Basic Materials,19.76,7.99,17.85,55.24,Fortress
10,DNG.TO,Basic Materials,5.87,8.56,17.85,52.04,Fortress
66,MX.TO,Basic Materials,54.44,8.88,17.85,50.25,Fortress
54,CIA.TO,Basic Materials,5.49,9.09,17.85,49.08,Fortress


In [31]:
# ==========================================
# 2. ANALYST FILTER FUNCTION FOR INSIDER PICKS (NEW)
# ==========================================
def filter_for_analyst_ratings(Fortress_Burry_EV_EBITDA, max_score=2.5):
    """
    Fetches analyst data for the insider winners and filters for 'Buy' or better.
    Scale: 1.0 = Strong Buy, 5.0 = Sell.
    Cutoff: 2.5 ensures we get 'Buy' and 'Strong Buy'.
    """
    if Fortress_Burry_EV_EBITDA.empty:
        return Fortress_Burry_EV_EBITDA

    tickers = Fortress_Burry_EV_EBITDA["Ticker"].tolist()

    try:
        yq = Ticker(tickers, asynchronous=True)
        # 'financial_data' contains the specific recommendation scores
        fin_data = yq.financial_data

        analyst_data = []
        for t in tickers:
            # Check if we got valid data for this ticker
            if isinstance(fin_data, dict) and t in fin_data:
                data = fin_data[t]
                # Ensure it's a dictionary and has the key we need
                if isinstance(data, dict) and "recommendationMean" in data:
                    score = data.get("recommendationMean")

                    # Only keep valid scores (sometimes they are None)
                    if score is not None:
                        analyst_data.append(
                            {
                                "Ticker": t,
                                "Analyst_Score": score,
                                "Analyst_Verdict": data.get("recommendationKey", "N/A"),
                            }
                        )

        df_analyst = pd.DataFrame(analyst_data)

        if df_analyst.empty:
            print("⚠️ No Analyst ratings found for these tickers.")
            return Fortress_Burry_EV_EBITDA  # Return original if no data found

        # Merge with the Fortress DataFrame
        merged = pd.merge(
            Fortress_Burry_EV_EBITDA, df_analyst, on="Ticker", how="inner"
        )

        # FILTER: Keep only scores <= max_score (Lower is better)
        final_df = merged[merged["Analyst_Score"] <= max_score].copy()

        print(
            f"✅ Analyst Filter: {len(merged)} -> {len(final_df)} stocks (Min Rating: Buy)."
        )
        return final_df.sort_values(by="Analyst_Score", ascending=True)

    except Exception as e:
        print(f"❌ Error in Analyst Filter: {e}")
        return Fortress_Burry_EV_EBITDA


# ==========================================
# 3. EXECUTION PIPELINE
# ==========================================

# A. Setup Tickers
if "Fortress_Burry_EV_EBITDA" in locals() and not Fortress_Burry_EV_EBITDA.empty:
    target_tickers = Fortress_Burry_EV_EBITDA["Ticker"].tolist()


# B. Run Insider Filter
Fortress_Burry_EV_EBITDA = filter_burry_ev_ebitda(fortress_df)

# C. Run Analyst Filter (NEW STEP)
# We overwrite 'Fortress_insiders' so it works with your Data Wrangler flow
if not Fortress_Burry_EV_EBITDA.empty:
    Fortress_Burry_Analyst_buy = filter_for_analyst_ratings(
        Fortress_Burry_EV_EBITDA, max_score=2.5
    )
else:
    Fortress_Burry_Analyst_buy = pd.DataFrame()

# D. Display Result
if not Fortress_Burry_Analyst_buy.empty:
    print(
        f"\n🚀 Final List: {len(Fortress_Burry_Analyst_buy)} stocks (Fortress + Burry + Analyst Buy Rating)"
    )
    display(Fortress_Burry_Analyst_buy)
else:
    print("No stocks passed all filters.")

📉 Analyzing EV/EBITDA for 74 Fortress stocks...

--- 📊 SECTOR AVERAGES (EV/EBITDA) ---
            Sector  Sector_Avg_EV_EBITDA
   Basic Materials                 17.85
 Consumer Cyclical                 15.72
Consumer Defensive                 12.14
            Energy                  7.43
       Industrials                 13.60
        Technology                 17.97
         Utilities                 10.87
✅ Analyst Filter: 25 -> 24 stocks (Min Rating: Buy).

🚀 Final List: 24 stocks (Fortress + Burry + Analyst Buy Rating)


,Ticker,Tier,Price,P/E,Sector,Z-Score,ROIC %,Op Margin %,Avg Margin (4Y),Curr Ratio,Int Cov,Mkt Cap (B),EV/EBITDA,Sector_Avg_EV_EBITDA,Discount_%,Analyst_Score,Analyst_Verdict
23,SVM.TO,Fortress,11.48,76.53,Basic Materials,6.25,10.35,42.70,30.35,4.588,22.11,2.53,15.97,17.85,10.53,1.33333,strong_buy
3,MX.TO,Fortress,54.44,13.34,Basic Materials,1.93,7.03,8.61,10.84,2.087,3.11,4.21,8.88,17.85,50.25,1.44444,strong_buy
18,BDI.TO,Fortress,14.66,25.28,Industrials,2.01,8.05,14.56,12.08,1.352,3.62,0.99,11.21,13.60,17.57,1.50000,strong_buy
7,TXG.TO,Fortress,65.54,14.06,Basic Materials,8.91,19.55,43.61,32.65,1.377,100.00,6.30,10.68,17.85,40.17,1.50000,strong_buy
12,CVE.TO,Fortress,23.22,13.42,Energy,2.60,9.60,11.00,10.14,1.729,7.27,43.80,5.50,7.43,25.98,1.52941,buy
5,OGC.TO,Fortress,38.90,16.62,Basic Materials,10.78,12.37,36.34,20.19,1.274,12.14,8.89,9.99,17.85,44.03,1.54545,buy
6,PHX.TO,Fortress,7.50,6.94,Energy,3.28,24.67,2.00,8.13,1.901,32.76,0.34,4.33,7.43,41.72,1.66667,buy
14,ABX.TO,Fortress,59.79,20.98,Basic Materials,4.85,11.18,49.01,30.18,2.944,11.89,102.00,13.49,17.85,24.43,1.68182,buy
4,CIA.TO,Fortress,5.49,24.95,Basic Materials,2.58,11.14,26.77,33.56,2.568,9.17,2.98,9.09,17.85,49.08,1.68750,buy
24,AEM.TO,Fortress,232.76,24.87,Basic Materials,8.41,10.21,53.10,29.01,2.120,33.63,116.93,16.66,17.85,6.67,1.75000,buy


In [36]:
# ==========================================
# 💎 THE "DEEP VALUE" INTERSECTION (Buffett + Burry)
# ==========================================

# 1. Ensure we have the Buffett Data
# (If you haven't run the Buffett scanner in this notebook yet, this runs it now)
if "Buffett_Value_DF" not in locals():
    print("🔄 Buffett Data not found. Running scan now...")
    if "get_buffett_value_picks" in globals() and "final_results" in locals():
        Buffett_Value_DF = get_buffett_value_picks(final_results)
    else:
        print("⚠️ Missing 'final_results' or 'get_buffett_value_picks' function.")
        Buffett_Value_DF = pd.DataFrame()

# 2. Ensure we have the Burry Data
if "Fortress_Burry_EV_EBITDA" not in locals():
    print("⚠️ Please run the Burry EV/EBITDA filter cell first.")
    Fortress_Burry_EV_EBITDA = pd.DataFrame()

# 3. THE MERGE (Finding the Overlap)
if not Buffett_Value_DF.empty and not Fortress_Burry_EV_EBITDA.empty:

    # Merge on Ticker to find stocks that appear in BOTH lists
    # We use an 'inner' join, which means "keep only if in both"
    Deep_Value_Intersection = pd.merge(
        Buffett_Value_DF[["Ticker", "P/B Ratio", "ROE %", "Debt/Eq %"]],
        Fortress_Burry_EV_EBITDA[
            [
                "Ticker",
                "Price",
                "Sector",
                "EV/EBITDA",
                "Sector_Avg_EV_EBITDA",
                "Discount_%",
                "Tier",
            ]
        ],
        on="Ticker",
        how="inner",
    )

    if not Deep_Value_Intersection.empty:
        print("\n" + "=" * 60)
        print(f"💎 DEEP VALUE GEMS FOUND: {len(Deep_Value_Intersection)}")
        print("=" * 60)
        print(
            "Criteria: Trading < Book Value (Buffett) AND Cheaper than Sector (Burry)"
        )

        # Sort by the "Discount" (how cheap they are vs sector)
        Deep_Value_Intersection = Deep_Value_Intersection.sort_values(
            by="Discount_%", ascending=False
        )

        cols = [
            "Ticker",
            "Price",
            "Tier",
            "P/B Ratio",
            "EV/EBITDA",
            "Sector_Avg_EV_EBITDA",
            "Discount_%",
            "Sector",
        ]
        display(Deep_Value_Intersection[cols])

        # --- SAVE TO FOLDER (Using the variable from Cell 2) ---
        # If DEEPVAL_CSV isn't defined, fallback to a safe default
        save_path = (
            DEEPVAL_CSV if "DEEPVAL_CSV" in locals() else "Deep_Value_Gems_CA.csv"
        )

        Deep_Value_Intersection.to_csv(save_path, index=False)
        print(f"\n✅ Successfully saved to: '{save_path}'")
        # -------------------------------------------------------
    else:
        print("\n❌ No stocks passed BOTH filters.")
        print(
            "This means no stock is both 'Below Book Value' AND 'Cheaper than Sector Average' at the same time."
        )
        print(
            f"Buffett Count: {len(Buffett_Value_DF)} | Burry Count: {len(Fortress_Burry_EV_EBITDA)}"
        )

else:
    print("❌ Cannot combine. One of the filters returned 0 results.")


💎 DEEP VALUE GEMS FOUND: 1
Criteria: Trading < Book Value (Buffett) AND Cheaper than Sector (Burry)


,Ticker,Price,Tier,P/B Ratio,EV/EBITDA,Sector_Avg_EV_EBITDA,Discount_%,Sector
0,TCL-A.TO,22.72,Fortress,0.99,6.58,15.72,58.14,Consumer Cyclical



✅ Successfully saved to: 'YfinanceDataDump\Deep_Value_Gems_CA.csv'


In [33]:
# ==========================================
# Watchlist Combiner (Finviz + YFinance)
# ==========================================


import pandas as pd
import yfinance as yf
from finvizfinance.quote import finvizfinance
import time
import numpy as np

# --- 1. INPUT YOUR MANUAL LIST HERE ---
MY_TICKERS = [
    "GRND",
    "ARCC",
    "BANC",
    "ONB",
    "UBER",
    "ADMA",
    "MIR",
    "APG",
    "SEI",
    "FLEX",
    "DD",
    "SVM",
]


def get_combined_watchlist(ticker_list):
    print(f"--- Processing {len(ticker_list)} stocks ---")

    # --- PART A: Get Analyst Ratings from Finviz ---
    print("1. Fetching Analyst Ratings from Finviz...")
    finviz_data = []

    for ticker in ticker_list:
        try:
            stock = finvizfinance(ticker)
            info = stock.ticker_fundament()

            finviz_data.append(
                {
                    "Ticker": ticker,
                    "Recom": info.get("Recom", np.nan),
                    "Target_Price": info.get("Target Price", np.nan),
                }
            )
            time.sleep(0.5)

        except Exception as e:
            print(f"   Skipping Finviz for {ticker}: {e}")
            finviz_data.append(
                {"Ticker": ticker, "Recom": np.nan, "Target_Price": np.nan}
            )

    df_finviz = pd.DataFrame(finviz_data)

    # --- PART B: Get Real-Time Stats from yfinance ---
    print("2. Fetching Price & Volatility from yfinance...")

    try:
        # Download data (1 Year is perfect for 52-Week MA)
        data = yf.download(
            ticker_list,
            period="1y",
            interval="1d",
            group_by="ticker",
            progress=False,
            threads=True,
        )
        yf_stats = []

        for ticker in ticker_list:
            try:
                # --- FIXED: Robust Data Extraction ---
                if isinstance(data.columns, pd.MultiIndex):
                    if ticker in data.columns.levels[0]:
                        df = data[ticker].copy()
                    else:
                        print(f"   Warning: {ticker} not found in yfinance download.")
                        continue
                else:
                    df = data.copy()

                # Cleanup
                df = df.dropna(subset=["Close"])
                if len(df) < 20:
                    print(f"   Warning: Not enough data for {ticker}")
                    continue

                # --- MATH CALCULATIONS ---
                current_price = df["Close"].iloc[-1]
                prev_close = df["Close"].iloc[-2]

                high_52 = df["High"].max()
                drop_from_high = ((current_price - high_52) / high_52) * 100

                change_pct = ((current_price - prev_close) / prev_close) * 100

                # Volatility (30-day Std Dev)
                volatility = df["Close"].pct_change().std() * 100

                # Relative Volume
                curr_vol = df["Volume"].iloc[-1]
                avg_vol = df["Volume"].tail(30).mean()
                rel_vol = curr_vol / avg_vol if avg_vol > 0 else 0

                # --- NEW: 52-Week Moving Average ---
                # Since we fetched exactly 1 year ('1y'), the mean of the whole column is the 52W MA
                ma_52w = df["Close"].mean()

                # Distance from MA (Optional but helpful metric)
                # dist_ma = ((current_price - ma_52w) / ma_52w) * 100

                yf_stats.append(
                    {
                        "Ticker": ticker,
                        "Price": round(current_price, 2),
                        "Change_%": round(change_pct, 2),
                        "52W_MA": round(ma_52w, 2),  # <--- Added Here
                        "Drop_from_High_%": round(drop_from_high, 2),
                        "Volatility_%": round(volatility, 2),
                        "Rel_Volume": round(rel_vol, 2),
                    }
                )

            except Exception as e:
                print(f"   Error calculating stats for {ticker}: {e}")
                continue

        df_yf = pd.DataFrame(yf_stats)

    except Exception as e:
        print(f"yfinance Critical Error: {e}")
        return pd.DataFrame()

    # --- PART C: Merge ---
    if not df_finviz.empty:
        if not df_yf.empty:
            master_df = pd.merge(df_finviz, df_yf, on="Ticker", how="outer")
        else:
            master_df = df_finviz

        # Added '52W_MA' to this list so it displays in the final table
        cols = [
            "Ticker",
            "Price",
            "Change_%",
            "52W_MA",
            "Drop_from_High_%",
            "Recom",
            "Target_Price",
            "Rel_Volume",
            "Volatility_%",
        ]

        final_cols = [c for c in cols if c in master_df.columns]
        return master_df[final_cols]
    else:
        return pd.DataFrame()


# --- RUN IT ---
watchlist_df = get_combined_watchlist(MY_TICKERS)

if not watchlist_df.empty:
    if "Drop_from_High_%" in watchlist_df.columns:
        watchlist_df["Drop_from_High_%"] = pd.to_numeric(
            watchlist_df["Drop_from_High_%"], errors="coerce"
        )
        print("\n--- Final Watchlist ---")
        display(watchlist_df.sort_values(by="Drop_from_High_%", ascending=True))
    else:
        display(watchlist_df)
else:
    print("No data found.")

--- Processing 12 stocks ---
1. Fetching Analyst Ratings from Finviz...
2. Fetching Price & Volatility from yfinance...

--- Final Watchlist ---


,Ticker,Price,Change_%,52W_MA,Drop_from_High_%,Recom,Target_Price,Rel_Volume,Volatility_%
6,GRND,13.54,0.97,17.62,-46.12,1.40,21.75,0.83,3.18
0,ADMA,18.24,-0.65,17.91,-28.94,1.00,30.00,0.76,3.26
7,MIR,23.42,-1.18,19.97,-22.65,1.12,30.62,0.71,3.49
11,UBER,81.71,-0.50,84.70,-19.88,1.47,112.40,0.42,2.39
9,SEI,45.97,-0.28,32.67,-19.41,1.17,65.45,0.69,5.90
5,FLEX,60.42,-2.03,48.48,-16.34,1.50,76.00,0.31,2.91
2,ARCC,20.23,-0.30,20.48,-9.64,1.27,22.64,0.89,1.38
10,SVM,8.34,-2.80,4.91,-9.05,1.17,9.43,0.57,3.62
8,ONB,22.31,-1.33,21.41,-6.56,1.85,25.92,0.76,2.14
1,APG,38.26,-1.54,31.52,-5.72,1.45,43.40,1.01,1.93


In [34]:
import requests
import pandas as pd
import google as genai
import enum
from typing_extensions import TypedDict
import json
import plotly.express as px
import sys

#!"{sys.executable}" -m pip install google.genai
#!"{sys.executable}" -m pip install plotly.express

In [35]:
tickers_gemini = ["TCL-A.TO"]
import os
from google import genai
from google.genai import types
from IPython.display import display, Markdown

# ==========================================
# SECURE CONFIGURATION
# ==========================================

# 1. Define the path to your key file
# If the file is in the same folder as this notebook, just use the filename.
KEY_FILE_PATH = (
    "C:\\Users\\James\\OneDrive - McMaster University\\Gemini API Key\\gemini_key.txt"
)


def load_api_key(filepath):
    """
    Reads the API key from a local file to avoid hardcoding it.
    """
    try:
        with open(filepath, "r") as f:
            # .strip() removes any accidental newlines or spaces
            return f.read().strip()
    except FileNotFoundError:
        print(f"❌ Error: Could not find the file '{filepath}'")
        print("Please create a text file with your API key in it.")
        return None
    except Exception as e:
        print(f"❌ Error reading key file: {e}")
        return None


# 2. Load the key and set the environment variable
api_key = load_api_key(KEY_FILE_PATH)

if api_key:
    os.environ["GEMINI_API_KEY"] = api_key
    print("✅ API Key loaded securely.")
else:
    print("⚠️ CRITICAL: API Key not loaded. The script will fail.")


# ==========================================
# SENTIMENT ANALYSIS FUNCTION
# ==========================================
def analyze_sentiment_gemini_3(tickers_gemini, company_name=None):

    if not os.environ.get("GEMINI_API_KEY"):
        print("❌ Stop: No API Key found.")
        return

    print(
        f"\n🧠 Gemini 3 is thinking (High Reasoning Mode)... analyzing ${tickers_gemini}..."
    )

    # Initialize Client
    client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

    config = types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(
            include_thoughts=False, thinking_level="HIGH"
        ),
        tools=[types.Tool(google_search=types.GoogleSearch())],
        response_modalities=["TEXT"],
    )

    prompt = f"""
    You are a Senior Equity Research Analyst using the Gemini 3 Reasoning Engine. 
    Perform a deep "Market Sentiment Analysis" on {tickers_gemini} ({company_name if company_name else 'the company'}).
    
    Step 1: SEARCH. Use Google Search to find the latest (last 30 days) news, analyst notes, and SEC filings.
    Step 2: REASON. Analyze the search results to determine the true market psychology. Look for contradictions between price action and news.
    
    Investigate these 4 Pillars:
    1. **News Virality**: Are headlines fear-mongering or euphoric? (Look for scandals, lawsuits, or product breakthroughs).
    2. **Analyst Shifts**: Are price targets moving UP or DOWN in the last week?
    3. **Institutional Flows**: Any reports of hedge funds or insiders buying/selling?
    4. **The "Whisper" Number**: What are traders saying on forums vs. official guidance?

    **OUTPUT FORMAT:**
    Produce a professional Markdown report:
    
    ## 🧠 Gemini 3 Sentiment Report: {tickers_gemini}
    **Reasoning Depth:** High
    **Sentiment Score:** [1-10]
    **Verdict:** [Buy / Hold / Sell / Speculative]
    
    ### 1. The Bull Thesis (Why it goes up)
    * ...
    
    ### 2. The Bear Thesis (Why it goes down)
    * ...
    
    ### 3. Deep Dive Analysis
    * **News Analysis**: ...
    * **Smart Money**: ...
    * **Financial Statement Analysis**: (Historic performance over last 3 years + expected performance)
    
    ### 4. Conclusion
    [Summary of whether the current price is a trap or an opportunity]
    """

    try:
        response = client.models.generate_content(
            model="gemini-3-flash-preview",  # Or 'gemini-3-flash-preview'
            contents=prompt,
            config=config,
        )
        display(Markdown(response.text))

    except Exception as e:
        print(f"❌ Error: {e}")


# ==========================================
# EXECUTION
# ==========================================
# Only run this if the key loaded successfully
if os.environ.get("GEMINI_API_KEY"):
    analyze_sentiment_gemini_3(tickers_gemini, tickers_gemini)

✅ API Key loaded securely.

🧠 Gemini 3 is thinking (High Reasoning Mode)... analyzing $['TCL-A.TO']...


## 🧠 Gemini 3 Sentiment Report: Transcontinental Inc. (TCL-A.TO)
**Reasoning Depth:** High  
**Sentiment Score:** 8.5/10 (Highly Bullish / Special Situation)  
**Verdict:** Speculative Buy (Arbitrage / Value Unlock)

---

### 1. The Bull Thesis (Why it goes up)
*   **Massive Capital Return**: The company has entered a definitive agreement (Dec 8, 2025) to sell its entire Packaging Sector to ProAmpac for **$2.22 billion CAD**. Management has signaled an intention to distribute approximately **$20.00 per share** to shareholders in early 2026.
*   **Stub Value Mispricing**: With the stock trading around **$22.72**, the market is valuing the remaining business—Canada’s largest printer and a major educational publisher—at roughly **$2.72 per share**. This legacy business generated significant cash flow in 2025, even with revenue headwinds.
*   **Operational Efficiency**: Adjusted EPS grew **10.7%** in FY 2025 despite a slight revenue decline, proving that the company's "profitability improvement program" is effectively lean-sizing the organization.
*   **Contract Stability**: The 10-year renewal of the printing contract for *The Globe and Mail* provides a long-term revenue floor for the "stub" printing business.

### 2. The Bear Thesis (Why it goes down)
*   **Execution Risk**: The $20.00 distribution is contingent on the deal closing in Q1 2026. Any regulatory hurdles or "material adverse change" clauses triggered by the buyer (ProAmpac) would cause the stock to collapse toward its pre-announcement levels.
*   **Secular Decline in Printing**: The legacy printing business faces terminal decline. The Canada Post labor disruptions in late 2025 highlighted the vulnerability of the flyer distribution model (raddar).
*   **Tax Implications**: For retail investors, the $20.00 distribution may be treated as a taxable dividend or a return of capital, depending on the final structure, which could lead to selling pressure from certain tax-sensitive accounts.

---

### 3. Deep Dive Analysis

#### **News Analysis**
Headline sentiment is currently **euphoric but skeptical**. While the $20.00 per share figure is widely reported, the fact that the stock is trading only slightly above that level suggests the market is pricing in a "wait-and-see" premium. There is no major fear-mongering regarding lawsuits, but the recent **Canada Post strike** dominated headlines in Dec 2025, providing a convenient excuse for the weaker Q4 printing volumes.

#### **Smart Money**
*   **Institutional Flows**: Historically, TCL-A has had relatively low institutional ownership (~25%). "Smart money" has been cautious due to the debt taken on for the packaging expansion. However, the divestiture effectively turns the company into a cash-rich shell plus a printing business, which is starting to attract "Special Situation" hedge funds.
*   **Analyst Shifts**: Consensus price targets remain around **$27.00**. There is a massive contradiction between these targets and the current price; analysts are valuing the "post-dividend stub" much higher than the current $2.72 implied market value.

#### **Financial Statement Analysis**
*   **Historic Performance (3-Year):**
    *   **2023-2024**: Focused on cost-cutting and debt reduction. Revenue stayed flat at ~$2.8B.
    *   **2025**: Revenue dipped to $2.7B, but **Adjusted EBITDA margins improved** due to the divestiture of lower-margin industrial packaging and optimization of the retail sector.
*   **Expected Performance**: Post-divestiture, Transcontinental will be a much smaller, high-yield company. Net indebtedness decreased by **$3.1M in interest expenses** in Q4 2025 alone, suggesting a very clean balance sheet once the $2.1B in proceeds hits the books.

---

### 4. Conclusion
**Opportunity.** The current price of **$22.72** is a classic "special situation" mispricing. If the $2.22B packaging sale closes as expected in Q1 2026, investors are essentially buying the largest printer in Canada for less than **$3.00 per share**. While the printing business is in secular decline, its cash-flow generation and renewed 10-year contracts make it worth significantly more than the current "stub" valuation.

**Traders should watch the Jan 2026 special shareholder meeting closely.** If the deal is approved, the gap between the current price and the total value (Cash + Stub) should close rapidly.